In [4]:
import json
import pandas as pd
import requests
import numpy as np

## FRANCE
def download_fra_data():
  url = "https://www.data.gouv.fr/fr/datasets/r/54dd5f8d-1e2e-4ccb-8fb8-eac68245befd"
  data = requests.get(url)

  with open('../data/input/vacsi-a-fra.csv', 'wb') as f:
          f.write(data.content)
          
def prepare_data(df):
  df=df[df["clage_vacsi"] != 0]
  df_clage_vacsi = pd.read_csv('../data/input/clage_spf.csv', sep=';')
  df = df.merge(df_clage_vacsi, left_on="clage_vacsi", right_on="code_spf").groupby(["jour", "categorie_fine"]).sum().reset_index()
  return df

def import_fra_data():
  df = pd.read_csv('../data/input/vacsi-a-fra.csv', sep=None, engine='python')
  return prepare_data(df)

def csv_to_json_fra(df):
  dict_json = {}

  for clage in df["categorie_fine"].tolist():
    df_clage = df[df["categorie_fine"] == clage]
    dict_json[clage] = {
                "jour": df_clage.jour.tolist(),
                "n_dose1": df_clage.n_dose1.tolist(), 
                "n_dose1_cum_pop": list(np.round(df_clage.n_cum_dose1.values/df_clage.population.values*100, 2))
              }

  with open("data/output/vacsi-tot-a-fra.json", "w") as outfile:
    outfile.write(json.dumps(dict_json))


download_fra_data()
df = import_fra_data()
print(df)
print(df[df.clage_vacsi>=59].n_dose1.sum())
#csv_to_json_fra(df)


            jour  categorie_fine  clage_vacsi  n_dose1  n_complet  \
0     2020-12-27     18 - 24 ans           24        0          0   
1     2020-12-27     25 - 29 ans           29        0          0   
2     2020-12-27     30 - 39 ans           39        0          0   
3     2020-12-27     40 - 49 ans           49        1          0   
4     2020-12-27     50 - 59 ans           59        2          0   
...          ...             ...          ...      ...        ...   
1235  2021-04-29      60 -64 ans           64    72131      19566   
1236  2021-04-29     65 - 69 ans           69    68771      23714   
1237  2021-04-29     70 - 74 ans           74    37734      52734   
1238  2021-04-29     75 - 79 ans           79    13180      20001   
1239  2021-04-29  80 ans et plus           80    20494      34036   

      n_cum_dose1  n_cum_complet  couv_dose1  couv_complet  code_spf  \
0               0              0         0.0           0.0        24   
1               0          

In [10]:
df_lastday = df[df.jour == df.jour.max()]
df_lastday[df_lastday.clage_vacsi >= 59].n_cum_dose1.sum() / df_lastday[df_lastday.clage_vacsi >= 59].population.sum()

0.5156708333333333